In [1]:
from opensearchpy import OpenSearch
from sentence_transformers import SentenceTransformer

/home/puju/py_playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# OpenSearch 클라이언트 연결
client = OpenSearch(
    hosts=[{'host': '172.30.1.81', 'port': 9200}],
    http_auth=None,
    use_ssl=False,
    verify_certs=False
)

In [3]:
# embedding 모델 불러오기
model = SentenceTransformer(
        "./model/KURE-v1",
        local_files_only=True,
)

Loading weights: 100%|██████████████████████████████████████████████████████████████| 391/391 [00:00<00:00, 401.63it/s, Materializing param=pooler.dense.weight]


In [9]:
# 인덱스 생성 (한국어 분석기 + 벡터 필드)
index_name = 'korean_test'

index_body = {
    "settings": {
        "index.knn": True,
        "analysis": {
            "tokenizer": {
                "nori_tokenizer": {
                    "type": "nori_tokenizer",
                    "decompound_mode": "mixed"
                }
            },
            "analyzer": {
                "korean": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "analyzer": "korean"
            },
            "embedding": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene"  # nmslib → lucene으로 변경
                }
            }
        }
    }
}

# 기존 인덱스 삭제 (테스트용)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

# 인덱스 생성
client.indices.create(index=index_name, body=index_body)
print(f"인덱스 '{index_name}' 생성 완료\n")

인덱스 'korean_test' 생성 완료



In [10]:
# 샘플 문서 색인
documents = [
    "봉명라이트핏",
    "봉명",
    "라이트핏",
    "라이트핏",
    "헬스",
    "충북헬스",
    "청주시헬스",
    "흥덕구헬스",
    "봉명동헬스",
    "충북헬스",
    "청주헬스",
    "흥덕헬스",
    "봉명헬스",
    "충북휘트니스",
    "청주시휘트니스",
    "흥덕구휘트니스",
    "봉명동휘트니스",
    "충북휘트니스",
    "청주휘트니스",
    "흥덕휘트니스",
    "봉명휘트니스",
    "충북피트니스",
    "청주시피트니스",
    "흥덕구피트니스",
    "봉명동피트니스",
    "충북피트니스",
    "청주피트니스",
    "흥덕피트니스",
    "봉명피트니스",
    "충북헬스장",
    "청주시헬스장",
    "흥덕구헬스장",
    "봉명동헬스장",
    "충북헬스장",
    "청주헬스장",
    "흥덕헬스장",
    "봉명헬스장",
    "충북",
    "청주시",
    "흥덕구",
    "봉명동",
    "충북",
    "청주",
    "흥덕",
    "봉명"
]

In [6]:
print("문서 색인 중...")
for i, text in enumerate(documents, 1):
    embedding = model.encode(text).tolist()
    print(len(embedding))
    
    client.index(
        index=index_name,
        id=i,
        body={
            "text": text,
            "embedding": embedding
        }
    )
    print(f"  [{i}] {text}")

문서 색인 중...
1024
  [1] 봉명라이트핏
1024
  [2] 봉명
1024
  [3] 라이트핏
1024
  [4] 라이트핏
1024
  [5] 헬스
1024
  [6] 충북헬스
1024
  [7] 청주시헬스
1024
  [8] 흥덕구헬스
1024
  [9] 봉명동헬스
1024
  [10] 충북헬스
1024
  [11] 청주헬스
1024
  [12] 흥덕헬스
1024
  [13] 봉명헬스
1024
  [14] 충북휘트니스
1024
  [15] 청주시휘트니스
1024
  [16] 흥덕구휘트니스
1024
  [17] 봉명동휘트니스
1024
  [18] 충북휘트니스
1024
  [19] 청주휘트니스
1024
  [20] 흥덕휘트니스
1024
  [21] 봉명휘트니스
1024
  [22] 충북피트니스
1024
  [23] 청주시피트니스
1024
  [24] 흥덕구피트니스
1024
  [25] 봉명동피트니스
1024
  [26] 충북피트니스
1024
  [27] 청주피트니스
1024
  [28] 흥덕피트니스
1024
  [29] 봉명피트니스
1024
  [30] 충북헬스장
1024
  [31] 청주시헬스장
1024
  [32] 흥덕구헬스장
1024
  [33] 봉명동헬스장
1024
  [34] 충북헬스장
1024
  [35] 청주헬스장
1024
  [36] 흥덕헬스장
1024
  [37] 봉명헬스장
1024
  [38] 충북
1024
  [39] 청주시
1024
  [40] 흥덕구
1024
  [41] 봉명동
1024
  [42] 충북
1024
  [43] 청주
1024
  [44] 흥덕
1024
  [45] 봉명


In [7]:
# 색인 완료 대기
client.indices.refresh(index=index_name)
print("\n색인 완료!\n")


색인 완료!



In [11]:
query_text = "청주 헬스장"

In [12]:
# 1. BM25 검색 (키워드 매칭)
print("\n[1] BM25 검색 결과:")
print("-" * 60)

bm25_query = {
    "query": {
        "match": {
            "text": query_text
        }
    }
}

bm25_results = client.search(index=index_name, body=bm25_query)

for hit in bm25_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['text']}")


[1] BM25 검색 결과:
------------------------------------------------------------
스코어: 1.8783 | 청주헬스장
스코어: 1.6529 | 청주시헬스장
스코어: 1.2862 | 청주헬스
스코어: 1.2068 | 충북헬스장
스코어: 1.2068 | 충북헬스장
스코어: 1.2068 | 흥덕헬스장
스코어: 1.2068 | 봉명헬스장
스코어: 1.1108 | 청주시헬스
스코어: 1.0620 | 흥덕구헬스장
스코어: 1.0620 | 봉명동헬스장


In [15]:
# 2. 벡터 검색 (의미 유사도)
print("\n[2] 벡터 검색 결과:")
print("-" * 60)

query_embedding = model.encode(query_text).tolist()

vector_query = {
    "query": {
        "knn": {
            "embedding": {
                "vector": query_embedding,
                "k": 5
            }
        }
    }
}

vector_results = client.search(index=index_name, body=vector_query)

for hit in vector_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['text']}")


[2] 벡터 검색 결과:
------------------------------------------------------------
스코어: 0.9960 | 청주헬스장
스코어: 0.9860 | 청주시헬스장
스코어: 0.9624 | 청주헬스
스코어: 0.9593 | 청주시헬스
스코어: 0.9239 | 충북헬스장


In [16]:
# 3. 하이브리드 검색 (BM25 + 벡터)
print("\n[3] 하이브리드 검색 결과:")
print("-" * 60)

hybrid_query = {
    "query": {
        "bool": {
            "should": [
                {
                    "match": {
                        "text": {
                            "query": query_text,
                            "boost": 0.5  # BM25 가중치
                        }
                    }
                },
                {
                    "knn": {
                        "embedding": {
                            "vector": query_embedding,
                            "k": 5,
                            "boost": 0.5  # 벡터 가중치
                        }
                    }
                }
            ]
        }
    }
}

hybrid_results = client.search(index=index_name, body=hybrid_query)

for hit in hybrid_results['hits']['hits']:
    print(f"스코어: {hit['_score']:.4f} | {hit['_source']['text']}")


[3] 하이브리드 검색 결과:
------------------------------------------------------------
스코어: 1.4371 | 청주헬스장
스코어: 1.3194 | 청주시헬스장
스코어: 1.1243 | 청주헬스
스코어: 1.0654 | 충북헬스장
스코어: 1.0350 | 청주시헬스
스코어: 0.6034 | 충북헬스장
스코어: 0.6034 | 흥덕헬스장
스코어: 0.6034 | 봉명헬스장
스코어: 0.5310 | 흥덕구헬스장
스코어: 0.5310 | 봉명동헬스장
